# Isolating Meshes from DTS

> DTS: Data of TotalSegmentator: https://zenodo.org/records/10047292

* DTS is about 23 Gbs which is large. It contains 1228 patients' CT and structures.
* The aim is to isolate one organ first (e.g. the liver) for training RegTR.
* Colab is quite fast for server-server transfer. Takes about 15 minutes to download the entire dataset with 26 Mb/s.

## Previous work on meshes

* https://github.com/sunyu0410/pytorch3d/tree/mesh/niimesh
  * Convert NIfTI to mesh (.obj)
  * Warp meshes using centre of mass
  * `voxelfy`, `to_mesh`, `NiiMesh`


## Download DTS

In [ ]:
!wget https://zenodo.org/records/10047292/files/Totalsegmentator_dataset_v201.zip?download=1

--2024-05-02 01:43:04--  https://zenodo.org/records/10047292/files/Totalsegmentator_dataset_v201.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23581218285 (22G) [application/octet-stream]
Saving to: ‘Totalsegmentator_dataset_v201.zip?download=1’

Totalsegmentator_da 100%[===================>]  21.96G  29.1MB/s    in 13m 0s  

2024-05-02 01:56:05 (28.8 MB/s) - ‘Totalsegmentator_dataset_v201.zip?download=1’ saved [23581218285/23581218285]



In [ ]:
!mv Totalsegmentator_dataset_v201.zip\?download\=1 Totalsegmentator_dataset_v201.zip

## Extraction

In [ ]:
import zipfile
from pathlib import Path

import shutil
import nibabel
import numpy as np
from tqdm import tqdm

In [ ]:
def extract_data(site, out_dir=None, in_zip='Totalsegmentator_dataset_v201.zip', total_n=1228):
  zip_ref = zipfile.ZipFile(in_zip, 'r')
  files = zip_ref.namelist()
  files_site = [i for i in files if Path(i).name.split('.')[0] == site]

  assert len(files_site) == total_n

  unzip_dir = Path('unzip')
  save_dir = Path(site) if out_dir is None else Path(out_dir)
  if not unzip_dir.exists(): unzip_dir.mkdir()
  if not save_dir.exists(): save_dir.mkdir()

  for f in files_site:
    zip_ref.extract(f, unzip_dir)

  # Check weather the contour is empty
  # If not, copy it; else keep note of the file
  empty = []
  for f in tqdm(files_site, desc=f'Copying {site}'):
    src = unzip_dir / f
    has_content = nibabel.load(src).get_fdata().astype(np.uint8).sum() > 0
    if has_content:
      dst = save_dir / f'{src.parents[1].name}_{site}.nii.gz'
      shutil.copy(src, dst)
    else:
      empty.append(f)

  return empty


In [ ]:
extract_data('liver')

1228


100%|██████████| 1228/1228 [02:15<00:00,  9.05it/s]


In [ ]:
extract_data('kidney_left')

1228


100%|██████████| 1228/1228 [02:13<00:00,  9.19it/s]


In [ ]:
extract_data('kidney_right')

1228


100%|██████████| 1228/1228 [02:21<00:00,  8.70it/s]


In [ ]:
!zip -qr /content/liver.zip liver
!zip -qr /content/kidney_left.zip kidney_left
!zip -qr /content/kidney_right.zip kidney_right

In [ ]:
sites = [
  'lung_lower_lobe_left',
  'lung_lower_lobe_right',
  'lung_middle_lobe_right',
  'lung_upper_lobe_left',
  'lung_upper_lobe_right',
  'skull',
  'brain',
]

In [ ]:
for site in sites:
  _ = extract_data(site)
  !zip -qr /content/{site}.zip {site}


Copying brain: 100%|██████████| 1228/1228 [02:15<00:00,  9.07it/s]


In [ ]:
!ls *.zip

brain.zip	  lung_lower_lobe_left.zip    lung_upper_lobe_right.zip
kidney_left.zip   lung_lower_lobe_right.zip   skull.zip
kidney_right.zip  lung_middle_lobe_right.zip  Totalsegmentator_dataset_v201.zip
liver.zip	  lung_upper_lobe_left.zip


In [ ]:
!mkdir structures

In [ ]:
!mv brain.zip	  lung_lower_lobe_left.zip    lung_upper_lobe_right.zip \
kidney_left.zip   lung_lower_lobe_right.zip   skull.zip \
kidney_right.zip  lung_middle_lobe_right.zip  liver.zip	  lung_upper_lobe_left.zip \
 structures

In [ ]:
!zip -qr structures.zip structures

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp structures.zip /content/drive/MyDrive/regtr_structures.zip

## Get an list of organs

In [ ]:
in_zip='Totalsegmentator_dataset_v201.zip'

In [ ]:
zip_ref = zipfile.ZipFile(in_zip, 'r')
files = zip_ref.namelist()

In [ ]:
temp = [Path(i) for i in files]

In [ ]:
temp[0]

PosixPath('s0000')

In [ ]:
temp2 = [i for i in temp if i.name.endswith('.nii.gz')]

In [ ]:
len(temp2) / 1228

118.0

In [ ]:
temp3 = set([i.name for i in temp2])

In [ ]:
temp3

{'adrenal_gland_left.nii.gz',
 'adrenal_gland_right.nii.gz',
 'aorta.nii.gz',
 'atrial_appendage_left.nii.gz',
 'autochthon_left.nii.gz',
 'autochthon_right.nii.gz',
 'brachiocephalic_trunk.nii.gz',
 'brachiocephalic_vein_left.nii.gz',
 'brachiocephalic_vein_right.nii.gz',
 'brain.nii.gz',
 'clavicula_left.nii.gz',
 'clavicula_right.nii.gz',
 'colon.nii.gz',
 'common_carotid_artery_left.nii.gz',
 'common_carotid_artery_right.nii.gz',
 'costal_cartilages.nii.gz',
 'ct.nii.gz',
 'duodenum.nii.gz',
 'esophagus.nii.gz',
 'femur_left.nii.gz',
 'femur_right.nii.gz',
 'gallbladder.nii.gz',
 'gluteus_maximus_left.nii.gz',
 'gluteus_maximus_right.nii.gz',
 'gluteus_medius_left.nii.gz',
 'gluteus_medius_right.nii.gz',
 'gluteus_minimus_left.nii.gz',
 'gluteus_minimus_right.nii.gz',
 'heart.nii.gz',
 'hip_left.nii.gz',
 'hip_right.nii.gz',
 'humerus_left.nii.gz',
 'humerus_right.nii.gz',
 'iliac_artery_left.nii.gz',
 'iliac_artery_right.nii.gz',
 'iliac_vena_left.nii.gz',
 'iliac_vena_right.nii.g

In [ ]:
len(temp3)

118

## Convert to Mesh

* Have problem running `pytorch3d` in Colab -> `pytorch3d.ops` can't be imported.
* PMCC cluster runs: `/physical_sciences/yusun/pytorch3d/env`

In [ ]:
!git clone https://github.com/sunyu0410/pytorch3d.git

Cloning into 'pytorch3d'...
remote: Enumerating objects: 12649, done.
remote: Counting objects: 100% (2190/2190), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 12649 (delta 2051), reused 2041 (delta 2041), pack-reused 10459
Receiving objects: 100% (12649/12649), 49.35 MiB | 34.95 MiB/s, done.
Resolving deltas: 100% (9084/9084), done.


In [ ]:
import os
os.chdir('pytorch3d')
!git checkout mesh

Branch 'mesh' set up to track remote branch 'mesh' from 'origin'.
Switched to a new branch 'mesh'


In [ ]:
!pip install /content/drive/MyDrive/py_wheel/pytorch3d-0.1.0-cp37-cp37m-linux_x86_64.whl

ERROR: pytorch3d-0.1.0-cp37-cp37m-linux_x86_64.whl is not a supported wheel on this platform.


In [ ]:
from pytorch3d import ops

ImportError: cannot import name 'ops' from 'pytorch3d' (/usr/local/lib/python3.10/dist-packages/pytorch3d/__init__.py)

In [ ]:
!os.chdir('skull')

/bin/bash: -c: line 1: syntax error near unexpected token `'skull''
/bin/bash: -c: line 1: `os.chdir('skull')'


In [ ]:
import torch